# Progress projection, immunity, ICU demand for COVID 19.
> Estimating and projecting - current and future percentage immunity per country, current and future need for ICUs beds, total cases from deaths, and cases and deaths per 100k.

- comments: true
- categories: [overview]
- author: artdgn
- image: images/covid-progress-projections.png
- permalink: /covid-progress-projections/
- hide: false

In [1]:
#hide
import pandas as pd
import overview_helpers

helper = overview_helpers.OverviewDataExtras
df = helper.filter_df(helper.table_with_projections())
df.columns

Index(['Cases.new', 'Cases.new.est', 'Cases.new.per100k',
       'Cases.new.per100k.est', 'Cases.total', 'Cases.total.est',
       'Cases.total.per100k', 'Cases.total.per100k.est', 'Deaths.new',
       'Deaths.new.per100k', 'Deaths.total', 'Deaths.total.per100k',
       'Fatality Rate', 'growth_rate', 'immune_ratio', 'immune_ratio.+14d',
       'immune_ratio.+30d', 'immune_ratio.+60d', 'needICU.per100k',
       'needICU.per100k.+14d', 'needICU.per100k.+30d', 'needICU.per100k.+60d'],
      dtype='object')

## Top 20 by immunisation progress: 
- With current reported fatality rate and current case growth rate.
- Sorted by number of new cases (estimated).


In [2]:
#hide_input
rename_cols = {'immune_ratio': 'Immune currently', 
               'immune_ratio.+14d': 'In 14 days', 
               'immune_ratio.+30d': 'In 30 days',
               'immune_ratio.+60d': 'In 60 days',
               'Fatality Rate': 'Reported <br> fatality rate',
               'growth_rate': 'Case growth <br> rate',
              }
progress_cols = list(rename_cols.values())[:4]
df_progress_bars = df.rename(rename_cols, axis=1)
df_progress_bars.sort_values('Cases.new.est', ascending=False)\
[rename_cols.values()]\
.head(20)\
    .style.bar(subset=progress_cols, color='#5fba7d', vmin=0, vmax=1.0)\
    .bar(subset=[rename_cols['Fatality Rate']], color='#420412', vmin=0, vmax=10)\
    .applymap(lambda _: 'color: red', subset=[rename_cols['Fatality Rate']])\
    .bar(subset=[rename_cols['growth_rate']], color='#d65f5f', vmin=1, vmax=2)\
    .set_precision(2).format('{:.1%}', subset=progress_cols)


,Immune currently,In 14 days,In 30 days,In 60 days,Reported fatality rate,Case growth rate
Country/Region,,,,,,
Spain,3.1%,15.9%,20.4%,20.9%,8.20,1.20
US,0.3%,19.3%,32.7%,32.8%,1.70,1.41
Italy,2.2%,8.3%,13.1%,14.7%,10.80,1.12
France,0.7%,6.0%,15.8%,18.5%,6.10,1.17
United Kingdom,0.4%,5.6%,18.4%,21.0%,5.90,1.21
Turkey,0.2%,44.5%,45.2%,45.2%,1.50,1.73
Iran,0.4%,1.1%,2.6%,5.9%,7.10,1.06
Netherlands,0.8%,7.3%,17.3%,19.4%,6.50,1.18
Belgium,0.8%,8.7%,19.3%,20.8%,3.90,1.20


## Top 20 by need for ICU beds per 100k with projections:
- With current new deaths burden (per 100k) and current case growth rate.

In [3]:
#hide_input
rename_cols = {'needICU.per100k': 'Current need <br> per 100k', 
               'needICU.per100k.+14d': 'In 14 days', 
               'needICU.per100k.+30d': 'In 30 days',
               'needICU.per100k.+60d': 'In 60 days',
               'Deaths.new.per100k': 'New deaths <br> per 100k',
               'growth_rate': 'Case growth <br> rate',
              }
icu_cols = list(rename_cols.values())[:4]
df_icu_bars = df.rename(rename_cols, axis=1)
df_icu_bars.sort_values(rename_cols['needICU.per100k'], ascending=False)\
[rename_cols.values()]\
.head(20)\
    .style.bar(subset=icu_cols, color='#f43d64', vmin=0, vmax=10)\
    .bar(subset=[rename_cols['Deaths.new.per100k']], color='#340849', vmin=0, vmax=10)\
    .applymap(lambda _: 'color: red', subset=[rename_cols['Deaths.new.per100k']])\
    .bar(subset=[rename_cols['growth_rate']], color='#d65f5f', vmin=1, vmax=2)\
    .set_precision(2).format('{:.2f}', subset=icu_cols)

,Current need per 100k,In 14 days,In 30 days,In 60 days,New deaths per 100k,Case growth rate
Country/Region,,,,,,
Spain,0.86,3.07,2.10,0.50,7.85,1.20
Italy,0.79,1.98,2.01,0.67,6.53,1.12
US,0.52,32.47,28.52,6.17,0.44,1.41
France,0.28,1.77,3.34,1.09,2.23,1.17
Iran,0.18,0.36,0.68,1.01,0.84,1.06
United Kingdom,0.18,1.81,4.26,1.26,1.01,1.21
China,0.13,0.13,0.14,0.14,0.00,1.00
Turkey,0.09,18.07,8.11,1.74,0.08,1.73
Netherlands,0.08,0.57,0.92,0.27,2.49,1.18


## Full overview and Need for ICU beds per 100K population, current and projected:
 - Sorted by current (estimated) need.
 - Only for countries with at least 10 deaths.

In [5]:
#hide_input
pretty_cols = {}

pretty_cols['deaths'] = 'Deaths (+new)'
df[pretty_cols['deaths']] =(df.apply(lambda r: f" \
                         {r['Deaths.total']:,.0f} \
                         (+<b>{r['Deaths.new']:,.0f}</b>) <br> \
                         Per 100k: {r['Deaths.total.per100k']:,.1f} \
                         (+<b>{r['Deaths.new.per100k']:,.1f}</b>) \
                         ", axis=1))

pretty_cols['cases'] = 'Cases (+new)'
df[pretty_cols['cases']] =(df.apply(lambda r: f" \
                         {r['Cases.total']:,.0f} \
                         (+<b>{r['Cases.new']:,.0f}</b>) <br>\
                         Est: {r['Cases.total.est']:,.0f} \
                         (+<b>{r['Cases.new.est']:,.0f}</b>)\
                         ", axis=1))

pretty_cols['icu'] = 'Need ICU <br>per 100k <br> (+ in 14/30/60 days)'
df[pretty_cols['icu']] =(df.apply(lambda r: f"\
                        <b>{r['needICU.per100k']:.2f}</b> <br>\
                        ({r['needICU.per100k.+14d']:.1f} / \
                        {r['needICU.per100k.+30d']:.1f} / \
                        {r['needICU.per100k.+60d']:.1f}) \
                        ", axis=1))

pretty_cols['progress'] = 'Immunized <br> percentage <br> (+ in 14/30/60 days)'
df[pretty_cols['progress']] =(df.apply(lambda r: f" \
                        <b>{r['immune_ratio']:.2%}</b> <br> \
                        ({r['immune_ratio.+14d']:.1%} / \
                        {r['immune_ratio.+30d']:.1%} / \
                        {r['immune_ratio.+60d']:.1%})", axis=1))

df.sort_values('needICU.per100k', ascending=False)\
    [pretty_cols.values()]\
    .style.set_na_rep("-").set_properties(**{})

,Deaths (+new),Cases (+new),Need ICU per 100k (+ in 14/30/60 days),Immunized percentage (+ in 14/30/60 days)
Country/Region,,,,
Spain,"5,982 (+3,671) Per 100k: 12.8 (+7.9)","73,235 (+38,099) Est: 1,431,140 (+744,521)",0.86 (3.1 / 2.1 / 0.5),3.06% (15.9% / 20.4% / 20.9%)
Italy,"10,023 (+3,946) Per 100k: 16.6 (+6.5)","92,472 (+28,545) Est: 1,314,192 (+405,675)",0.79 (2.0 / 2.0 / 0.7),2.17% (8.3% / 13.1% / 14.7%)
US,"2,026 (+1,469) Per 100k: 0.6 (+0.4)","121,478 (+77,631) Est: 859,417 (+549,214)",0.52 (32.5 / 28.5 / 6.2),0.26% (19.3% / 32.7% / 32.8%)
France,"2,317 (+1,455) Per 100k: 3.5 (+2.2)","38,105 (+17,982) Est: 461,517 (+217,793)",0.28 (1.8 / 3.3 / 1.1),0.71% (6.0% / 15.8% / 18.5%)
Iran,"2,517 (+705) Per 100k: 3.0 (+0.8)","35,408 (+12,359) Est: 302,562 (+105,608)",0.18 (0.4 / 0.7 / 1.0),0.36% (1.1% / 2.6% / 5.9%)
United Kingdom,"1,021 (+685) Per 100k: 1.5 (+1.0)","17,312 (+10,586) Est: 293,779 (+179,641)",0.18 (1.8 / 4.3 / 1.3),0.43% (5.6% / 18.4% / 21.0%)
China,"3,299 (+25) Per 100k: 0.2 (+0.0)","81,999 (+501) Est: 222,025 (+1,357)",0.13 (0.1 / 0.1 / 0.1),0.02% (0.0% / 0.1% / 0.1%)
Turkey,108 (+71) Per 100k: 0.1 (+0.1),"7,402 (+5,873) Est: 149,411 (+118,548)",0.09 (18.1 / 8.1 / 1.7),0.18% (44.5% / 45.2% / 45.2%)
Netherlands,640 (+426) Per 100k: 3.7 (+2.5),"9,819 (+5,055) Est: 139,680 (+71,910)",0.08 (0.6 / 0.9 / 0.3),0.82% (7.3% / 17.3% / 19.4%)


### Assumtions and references:
- Everything is pretty appoximate, I'm not an epidmiologist, just trying to get a guage of what's happening, how things are evolving, and what the future calculates to. The exact numbers may not be very important where differences between countries and policies have effects of multiple orders of magnitude.
- Total case estimation is done from deaths by:
    - Assuming that unbiased fatality rate is 1.5% (from heavily tested countries / Cruise ship) and that it takes 8 days on average for a case to go from being confirmed positive (after incubation + testing lag) to death. This is the same figure used by ["Estimating The Infected Population From Deaths"](https://covid19dashboards.com/covid-infected/) in this repo.
    - Calculating the testing bias (8 days ago), and applying that bias to current cases figures for that country.
- Projection is done using a simple SIR model with:
    - Growth rate calculated by ratio of new cases in 5 past days, to new cases in the 5 days before that. This is pessimmistic - because it doesn't weigh the recent days heavier and because includes the testing rate growth rate as well, so is slow to react to both improvements in test coverage and "flattenning".
    - Recovery probability being 1/20 (for 20 days to recover).
- ICU need is calculated as being [6% of active cases](https://medium.com/@joschabach/flattening-the-curve-is-a-deadly-delusion-eea324fe9727) where:
    - Active cases are taken from the SIR model (above).
    - This is both pessimmistic - because real ICU rate may in reality be lower, due to testing biases, and especially in "younger" populations), and optimistic - because active cases which are on ICU take longer (so need the ICUs for longer).
    - [Some numbers](https://www.forbes.com/sites/niallmccarthy/2020/03/12/the-countries-with-the-most-critical-care-beds-per-capita-infographic/) on actual capacity of ICUs per 100k (didn't find a full dataset for a lot of countries yet).